In [1]:
import csv
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
#Santiago et al. (2022) did not account for patterns of phonetic variation within his POS rules so we add them here
phonetic_variation = ["wanna", "tryna", "gonna", "gotta"]

In [3]:
corpus_path = "C:\\Users\\wiler\\Documents\\College\\Oral History\\features\\habitual be\\be_sentences (no labels).txt"
output_path = "C:\\Users\\wiler\\Documents\\College\\Oral History\\features\\habitual be\\Tagged Habitual Sentences.csv"

In [4]:
class Sentence:
    habituality = 0 #using a ternary system: -1 = nonhabitual 0 = unclassified, 1 = habitual
    pos1 = 0; pos2 = 0; pos3 = 0; pos4 = 0; pos5 = 0; pos6 = 0;
    synPar1 = 0; synPar2 = 0; synPar3 = 0; synPar4 = 0; 
    a1 = 0; a2 = 0; a3 = 0; a4 = 0; a5 = 0; r1 = 0;

    def __init__(self, text, num):
        self.text = text
        self.num = num

In [5]:
#Read sentences into the program
lines = []
with open(corpus_path, "r", encoding="utf_8", errors="ignore") as file:
    for line in lines:
        line = line.strip()
    lines = file.readlines()
    
print(len(lines))
print(lines)

9061
['Sentences\n', "Let's see- And, uh, if you didn't know already, my name is Quanette and I will be, the interviewer of the day.\n", '\n', 'It might be twenty- twenty-eighteen, before we have another one.\n', '\n', "It wasn't as big as I thought it would be.\n", '\n', 'It used to be so hot.\n', '\n', 'And I had- My major was gonna be nursing.\n', '\n', 'I knew it was gonna be math.\n', '\n', 'The two-thousands finna be considered old.\n', '\n', 'Ninety-six babies, is finna be twenty-one.\n', '\n', 'Gianfranco is about to be twenty-one in a little while.\n', '\n', 'Some of that stuff, you be like what?\n', '\n', 'Elysa be showing me some work.\n', '\n', 'Hey, but would you be able to do that without like that- that paper right in front of you.\n', '\n', 'So I just be like Oh, shit .\n', '\n', 'I just be liking the beat to a hip hop song.\n', '\n', 'I be listening to the beats.\n', '\n', 'Right, like And then I be seeing these videos.\n', '\n', "These mama's be letting they little bo

In [6]:
sen = [] #list of Sentence objects
for line in range(len(lines)):
    numBe = 0
    parsed = nlp(lines[line])
    for i, word in enumerate(parsed):
        if(word.text.lower() == "be"):
            numBe += 1
    if(numBe == 1):
        sen.append(Sentence(lines[line], line)) #creates Sentence object of every sentence with "be"

In [7]:
# Tags the Sentence objects with each rule
index = -1
senNum = 1
numBe = 0
extraBe = []
beIndices = []
count = 0

for Sentence in sen:
    print(Sentence.text)
    index += 1
    numBe = 0
    if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
        senNum += 1
    else:
        senNum = 1
    parsed = nlp(Sentence.text)
    for i, word in enumerate(parsed):
        if (word.text.lower() == "be"):
            numBe += 1
            if (numBe == senNum):
                count +=1
                beIndices.append(i)
                beChildren = list(word.children)
                if(word.head == word):
                    beSibling = []
                else:
                    beSibling = list(word.head.children)
                beDep = word.dep_
                bePOS = word.pos_
                beIndex = i
                preprepreprevious = None
                prepreprevious = None
                preprevious = None
                previous = None
                after = None
                postafter = None
                postpostafter = None
                postpostpostafter = None
                if(i >= 4):
                    preprepreprevious = parsed[i-4]
                if(i >= 3):
                    prepreprevious = parsed[i-3]
                if(i >= 2):
                    preprevious = parsed[i-2]
                if(i > 0):
                    previous = parsed[i-1]
                if(i+1 < len(parsed)):
                    after = parsed[i+1]
                if(i+2 < len(parsed)):
                    postafter = parsed[i+2]
                if(i+3 < len(parsed)):
                    postpostafter = parsed[i+3]
                if(i+4 < len(parsed)):
                    postpostpostafter = parsed[i+4]
                    
                #pos1: if the word immediately preceding "be" is a modal, adjective, or "to"
                if(previous and (previous.tag_ == "MD" or previous.tag_ == "JJ" or previous.tag_ == "TO")):
                    Sentence.pos1 = 1
                    Sentence.habituality = -1
                if(previous and previous.text in phonetic_variation):
                    Sentence.pos1 = 1
                    Sentence.habituality = -1
                #pos2: if the word immediately following "be" is an adjective, while the word immediately preceding "be" is not a personal pronoun nor a noun
                if ((after and previous) and (after.tag_ == "JJ" and previous.tag_ != "PRP" and not previous.tag_.__contains__("NN"))):
                    Sentence.pos2 = 1
                    Sentence.habituality = -1
                #pos3: if the word immediately following "be" is a preposition or subordinating conjunction, while the word immediately preceding "be" is a singular present verb
                if ((after and previous) and (after.tag_ == "IN" and (previous.tag_ == "VBZ" or previous.tag_ == "VBP"))):
                    Sentence.pos3 = 1
                    Sentence.habituality = -1
                #pos4: if the word immediately preceding "be" is a noun, and the word immediately preceding that noun is an adjective
                if (previous and preprevious) and (previous.tag_ == "NN" and preprevious.tag_ == "JJ"):
                    Sentence.pos4 = 1
                    Sentence.habituality = -1
                #pos5: if the word immediately preceding "be" is an adverb, and the word immediately following "be" is either a personal pronoun or determiner
                if (previous and after) and (previous.tag_ == "RB" and (after.tag_ == "PRP" or after.tag_ == "DT")):
                    Sentence.pos5 = 1
                    Sentence.habituality = -1
                #pos6: if the word immediately preceding "be" is an adverb, and either the word imemdiately preceding the adverb is a verb, or modal
                if (previous and preprevious) and (previous.tag_ == "RB" and preprevious.tag_.__contains__("VB") or preprevious.tag_ == "MD"):
                    Sentence.pos6 = 1
                    Sentence.habituality = -1

                #synPar1: if "be" is tagged as an auxiliary and its child has a dependency relation of auxiliary
                if len(beChildren) != 0:
                    for child in beChildren:
                        if child.dep_ == "aux" and child.pos_ == "AUX" and child.text.lower() != "do":
                            Sentence.synPar1 = 1
                            Sentence.habituality = -1
                #synPar2: if "be" is tagged as an auxiliary and its sibling has a dependency relation of auxiliary
                if len(beSibling) != 0:
                    for sibling in beSibling:
                        if sibling.dep_ == "aux" and sibling.pos_ == "AUX" and sibling.text.lower() != "do" and sibling.text != "be":
                            Sentence.synPar2 = 1
                            Sentence.habituality = -1
                #synPar3: if "be" is labeled as an auxiliary and has a head that is labeled as a verb
                if (word.pos_ == "AUX" and word.dep_ == "aux" and word.head.pos_ == "VERB"):
                    Sentence.synPar3 = 1
                    if(Sentence.habituality != -1):
                        Sentence.habituality = 1
                #synPar4: if "be" is labeled as a verb, contrasting its more frequent label of auxiliary
                if(word.pos_ == "VERB"):
                    Sentence.synPar4 = 1
                    Sentence.habituality = 1
                    
                #A1: "don't" preceding "be" indicates habituality except for in imperative sentences
                if (((previous and preprevious and prepreprevious) and (previous.text == "n't" and ((preprevious.text == "do") or (preprevious.text == "Do"))) and (prepreprevious.pos_ == "PRON" or prepreprevious.pos_ == "NOUN")) or ((preprevious and prepreprevious and preprepreprevious) and ((previous.pos_ != "VERB" and previous.pos_ != "AUX")) and preprevious.text == "n't" and ((prepreprevious.text == "do") or (prepreprevious.text == "Do")) and (preprepreprevious.pos_ == "PRON" or preprepreprevious.pos_ == "NOUN"))):
                    Sentence.a1 = 1
                    Sentence.habituality = 1
                #A2: when followed by interjections, conjunctions, determinants, proper nouns, and punctuation, "be" tends to be non-habitual
                if (((after) and (after.pos_ == "PUNCT" or after.pos_ == "CCONJ" or after.pos_ == "DET" or after.pos_ == "INTJ" or after.pos_ == "PROPN"))):
                    Sentence.a2 = 1
                    Sentence.habituality = -1
                #A3: if the "be" is preceded by a pronoun in the sentence, where the words between the pronoun and "be" are not auxiliaries, verbs, or particles, it tends to be habitual
                if ((previous and previous.pos_ == "PRON") or (preprevious and preprevious.pos_ == "PRON" and previous.pos_ != "AUX" and previous.pos_ != "VERB" and previous.pos_ != "PART") or (prepreprevious and prepreprevious.pos_ == "PRON" and previous.pos_ == "ADV" and preprevious.pos_ != "AUX" and preprevious.pos_ != "VERB")):
                    Sentence.a3 = 1
                    Sentence.habituality = 1


                #A4: "be" tends to be habitual when it is followed by a verb ending in -ing and is not preceded by an auxiliary verb, "to", or any of the words following words displaying phonetic variation of the infinitive "to": "gonna", "gotta", "wanna" or "tryna"
                if (after and after.tag_ == "VBG" and previous and previous.pos_ != "AUX" and not (previous.pos_ == "PART" and Sentence.pos1 == 1)):
                    Sentence.a4 = 1
                    Sentence.habituality = 1

                #A5: if "be" is preceded by a word ending in "n't" that is not "don't", it tends to be habitual
                if ((previous and preprevious) and (previous.text == "n't" and preprevious.text != "do" and preprevious.text != "Do")):
                    Sentence.a5 = 1
                    Sentence.habituality = -1
                    
                #R1 sentences that are not tagged by any POS rules tend to be habitual
                if ((Sentence.pos1 !=1) and (Sentence.pos2 !=1) and (Sentence.pos3 !=1) and (Sentence.pos4 !=1) and (Sentence.pos5 !=1) and (Sentence.pos6 !=1)):
                    Sentence.r1 = 1
                    Sentence.habituality = 1
            else:
                numBe -= 1

Let's see- And, uh, if you didn't know already, my name is Quanette and I will be, the interviewer of the day.

It might be twenty- twenty-eighteen, before we have another one.

It wasn't as big as I thought it would be.

It used to be so hot.

And I had- My major was gonna be nursing.

I knew it was gonna be math.

The two-thousands finna be considered old.

Ninety-six babies, is finna be twenty-one.

Gianfranco is about to be twenty-one in a little while.

Some of that stuff, you be like what?

Elysa be showing me some work.

Hey, but would you be able to do that without like that- that paper right in front of you.

So I just be like Oh, shit .

I just be liking the beat to a hip hop song.

I be listening to the beats.

Right, like And then I be seeing these videos.

These mama's be letting they little boys dance and stuff.

It going to be an interesting time to wonder.

Boy that'd be fire, if they all playing it together.

Obamacare be like, okay.

I be trying to either get too deep

In [8]:
#after conducting error analysis of the previous experiments, we discovered that some patterns interacted with each other in a way that indicated habituality beyond their indivdiual scopes
for Sentence in sen:
    #sentences that have been tagged by any POS rules are unlikely to be habitual, thus synpar3 (a rule that indicates habituality) is set to 0
    if (Sentence.r1 == 0): 
        Sentence.synPar3 = 0
    if (Sentence.a4 == 1): 
        Sentence.synPar3 = 0
    #PLEASE EXPLAIN THESE A BIT MORE
    if (Sentence.a2 == 1 and Sentence.a3 == 1):
        Sentence.pos5 = 0
    if (Sentence.a4 == 1):
        Sentence.a5 == 0
    if (Sentence.a3 == 1):
        Sentence.synPar2 = 0
    if (Sentence.pos6 == 1 or Sentence.synPar1 == 1):
        Sentence.a3 = 0
    if (Sentence.synPar1 == 1):
        Sentence.r1 = 0

In [9]:
#creates output spreadsheet
with open(output_path, mode="w", encoding="utf-8", newline="") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(["Habituality", "Sentence", "POS1", "POS2", "POS3", "POS4", "POS5", "POS6","A1", "A2","A3", "A4", "A5", "SynPar1", "SynPar2", "SynPar3", "SynPar4", "R1"])
    for Sentence in sen:
        filewriter.writerow([Sentence.habituality, Sentence.text, Sentence.pos1, Sentence.pos2, Sentence.pos3, Sentence.pos4, Sentence.pos5,Sentence.pos6, Sentence.a1, Sentence.a2, Sentence.a3, Sentence.a4, Sentence.a5, Sentence.synPar1, Sentence.synPar2, Sentence.synPar3, Sentence.synPar4, Sentence.r1])